In [2]:
# Importing notebook dependencies
%run FinanceTab.ipynb
# %run Twitter.ipynb @Kris -- input the notebooks you need to run for your Twitter functions
# %run News.ipynb @Satheesh -- input the notebooks you need to run for your News Analyses

# Note that some of the basic libraries are already included in the imported notebooks, so there was no need to re-import those libraries.
# These libraries include:
# import os
# import pandas as pd
# from datetime import datetime
# import matplotlib.pyplot as plt
# import panel as pn
# from panel.interact import interact
# from bokeh.models import HoverTool

# Initial imports
from eod import *
import requests
import datetime
from datetime import timedelta, date
from dateutil.parser import parse

# Import interactive panels libraries
from panel import widgets
pn.extension()

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, CustomJS, MultiSelect, CheckboxGroup, NumeralTickFormatter
from bokeh.models.widgets import Tabs, Panel

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
############################################################
"""
    Get Predictions Function
"""
############################################################

def get_predictions(%store -r ticker, %store -r search_word):
    """
    Technical Analysis, Fundamental Analysis, Twitter Sentiment, Article Sentiment
    Notebooks return the next 5 predicted price percent changes of a selected stock and save them into dataframes

    """
    
    # Run the Jupyter notebooks with user variables
    reg, info = trainModel(ticker)
    ta_predicted = predictPrices(ticker, reg, info)
    
    twitter_predicted = twitter_analysis(ticker, search_word)
    news_predicted = news_analysis(ticker)

    return ta_predicted, twitter_predicted, news_predicted
    
    
############################################################
"""
    Combine and Average Functions
"""
############################################################
    
def combine_average(ta_df, twitter_df, news_df):   
    
    last_close = ta_df["close"][0]/(1+ta_df["DR"][0])
    ta_df.drop(columns = "close", inplace = True)
    
    # Aligning Indices
    twitter_df.reset_index(drop = True, inplace = True)
    twitter_df.set_index(ta_df.index, inplace = True)
    
    news_df.reset_index(drop = True, inplace = True)
    news_df.set_index(ta_df.index, inplace = True)
    
    # Joins all the prediction dataframes into a single dataframe
    combined_df = pd.concat([ta_df, twitter_df, news_df], axis = "columns", join = "inner")
    
    # Averaging across the rows (axis = 1) of the combined dataframe
    avg_combined_predictions = pd.DataFrame(combined_df.mean(axis = 1), columns = ["Pred Returns"])
    
    # Calculating Prices from the Last Close
    avg_combined_predictions["Pred Close"] = -1.0
    
    for i in range(avg_combined_predictions.shape[0]):
        if i == 0:
            avg_combined_predictions["Pred Close"][i] = last_close*(1+avg_combined_predictions["Pred Returns"][i])
        else:
            avg_combined_predictions["Pred Close"][i] = avg_combined_predictions["Pred Close"][i-1]*(1+avg_combined_predictions["Pred Returns"][i])

    
    return avg_combined_predictions

In [5]:
############################################################
"""
    Main Function
"""
############################################################

# gets predictions and saves them to dataframes
ta_df, twitter_df, news_df = get_predictions(%store -r ticker, %store -r search_word) # (ticker and word will be retrieved from user inputs)

# combines and takes the average of all the predictions
avg_combined_predictions = combine_average(ta_df, twitter_df, news_df)

In [5]:
############################################################
"""
    Trade Recommendation Function
    
    A recommendation of 1.0 is to buy, -1.0 is to sell, 0.0 is to hold
"""
############################################################
def trade_reco(%store -r ticker, avg_combined_predictions):
    
    # Getting technical signals recommendation
    sig_rec = getSignalRec(ticker)
    
    # Calculating prediction recommendation
    pred_rec = 0.0
    open_price = avg_combined_predictions["Pred Close"].head(1)
    close_price = avg_combined_predictions["Pred Close"].tail(1)
    
    if close_price > open_price:
        pred_rec = 1.0
    elif close_price < open_price:
        pred_rec = -1.0
    
    # Sentiment recommendation
    sent_rec = 0.0 ## @Kris or Satheesh, how will you be making recommendations from sentiment?
    
    sum_rec = sig_rec + pred_rec + sent_rec
    
    if sum_rec > 0:
        trade_rec = 1.0
    elif sum_rec < 0:
        trade_rec = -1.0
    else:
        trade_rec = 0.0
        
    return trade_rec

In [ ]:
############################################################
"""
    Panel Dashboard Functions
"""
############################################################
############################################################
"""
    Define User Input Interaction
    Stores ticker and search_word inputs as global notebook variables
"""
############################################################

def user_inputs():
    
    %store ticker = pn.widgets.TextInput(name='Stock Ticker:')    
    %store search_word = pn.widgets.TextInput(name='Twitter Search')
    
    multi_select.js_on_change("value", 
                              CustomJS(
        code="""
        console.log('multi_select: value=' + this.value, this.toString())
        """)
                             )
    return %store ticker, %store search_words


############################################################
"""
    Define Prediction Plot
"""
############################################################

def price_prediction_graph(df):
    
    # Output the visualization directly in the notebook
    output_notebook()
    # Parse Column data
    source = ColumnDataSource(data=df)
    # Create Toolbar Selection
    select_tools = [HoverTool(), 'pan', 'wheel_zoom', 'tap', 'reset']
    
    tooltips = [
            ('Date', '@Date{%F}'),
            ('Adj. Close. Price', '@AdjClosePrice{$00.00}'),
           ]
    
    formatters = {
        '@Date' : 'datetime'
    }
    
    # Create the figure object
    predicted_fig = figure(
        x_axis_type='datetime',
        title=("Predicted Price Returns"),
        plot_height=500, plot_width=900,
        x_axis_label='Date', y_axis_label='Predicted Returns',
        toolbar_location='below',
        tools = select_tools
    )
    
    # Draw the figure
    predicted_fig.line(
        x='Date',
        y="predicted_change",
        source=source,
        color='green', 
        width=0.75, 
        legend_label='Price',
    )
       
    predicted_fig.legend.location = 'top_left'
    predicted_fig.add_tools(HoverTool(tooltips=tooltips, formatters=formatters))
    predicted_pp_pane = pn.pane.Bokeh(predicted_fig)
    
    return predicted_pp_pane

############################################################
"""
    Creating The Panes
"""
############################################################


# Creating the Price Prediction Pane
avg_predicted_price_pane = price_prediction_graph(avg_combined_predictions)

#Creating the Decision Recommendation Pane
trade_reco_pane = trade_reco(avg_combined_predictions)

# Creating the Individual Prediction Panes
ta_predicted_pane = price_prediction_graph(ta_df)
twitter_predicted_pane = price_prediction_graph(twitter_df)
news_predicted_pane = price_prediction_graph(news_df)

#Creating latest article & Tweet Panes
#latest_article_pane = 
#random_tweet_pane = 



In [ ]:
# Create Panel Dashboard

welcome_column = pn.Column(
    "# Welcome To The $4A Dashboard",
    "### We aim to maximize retail investors’ alpha over the long term by helping them make more informed decisions.",
    
    
    "#### Welcome to the asset analysis dashboard. Use the tabs above to navigate through the different sections."
    "Use the input below to see how your ideas and assets compare with the benchmarks.",
    
    "### Input the Stock Ticker and one search word you wish to view:",
    "##### *(Press and Hold Command to select multiple)*",
    user_inputs(),
    
    "# Predicted Price Returns",
    avg_predicted_price_pane,
    
    " # The Recommendation",
    trade_reco_pane,
)

individual_predictions = pn.Column(
     "# Predictions of Individual Analysis",
    
    "### Technical Analysis Predictions",
    ta_predicted_pane,
    "### Twitter Sentiment Analysis Predictions",
    twitter_predicted_pane,
    "### News Sentiment Analysis Predictions",
    news_predicted_pane,
)

news_tweets = pn.Column(
     "# Latest news article and random tweet about your stock:",
    
    "### Read The Latest News:",
    latest_article_pane,
    "### A Random Tweet For You:",
    random_tweet_pane,
)

finance_tab = getFinanceTab(%store -r ticker)

predictions_dashboard = pn.Tabs(
    (
        "Welcome",
        welcome_column
    ),
    (
        "Individual Analysis Predictions",
        individual_predictions
    ),
    (
        "Latest News",
        news_tweets
    ),
    (
        "Financial Information",
        finance_tab
    )
)

predictions_dashboard

In [ ]:
# show the dashboard with the data embedded

predictions_dashboard.embed()

In [ ]:
"""
To Run code in Terminal

First need to run predictions_dashboard.servable()

then in terminal, run
panel serve dashboard.ipynb --show
"""